In [7]:
# FIXED PATH DATA CLEANING SCRIPT - Run in VS Code Jupyter
# ========================================================
# This script fixes the path issue for CSV files

# Cell 1: Setup with Correct Paths
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import re
from datetime import datetime, timedelta
import os

# Fix path issue - go up one level from notebooks to project root
if 'notebooks' in str(Path.cwd()):
    project_root = Path.cwd().parent  # Go up one level from notebooks
else:
    project_root = Path.cwd()  # Already in project root

print(f"Project root: {project_root}")

# Data paths - CORRECTED
raw_data_path = project_root / "data" / "raw"
processed_data_path = project_root / "data" / "processed"

print(f"Raw data path: {raw_data_path}")
print(f"Processed data path: {processed_data_path}")

# Check if paths exist
print(f"Raw data path exists: {raw_data_path.exists()}")
print(f"Processed data path exists: {processed_data_path.exists()}")

# Create processed directory if it doesn't exist
processed_data_path.mkdir(parents=True, exist_ok=True)

# List files in raw data directory
if raw_data_path.exists():
    print(f"Files in raw data directory:")
    for file in raw_data_path.iterdir():
        print(f"  - {file.name}")
else:
    print("❌ Raw data directory not found!")

# Cell 2: Load Data with Correct Paths
print("\n📂 LOADING DATA WITH CORRECT PATHS")
print("="*50)

# Define exact file paths
csv_files = {
    'providers': raw_data_path / "providers_data.csv",
    'receivers': raw_data_path / "receivers_data.csv", 
    'food_listings': raw_data_path / "food_listings_data.csv",
    'claims': raw_data_path / "claims_data.csv"
}

# Check if all files exist
print("Checking CSV files:")
all_files_exist = True
for name, filepath in csv_files.items():
    if filepath.exists():
        print(f"✅ {name}: {filepath.name} found")
    else:
        print(f"❌ {name}: {filepath.name} NOT found")
        all_files_exist = False

if not all_files_exist:
    print("\n⚠️ Some CSV files are missing!")
    print("Make sure these files are in your data/raw/ folder:")
    print("- providers_data.csv")
    print("- receivers_data.csv") 
    print("- food_listings_data.csv")
    print("- claims_data.csv")
    
    # Alternative: try to find files with different names
    print("\nLooking for alternative file names...")
    if raw_data_path.exists():
        for file in raw_data_path.glob("*.csv"):
            print(f"Found: {file.name}")

# Load data with error handling
datasets = {}
try:
    providers_df = pd.read_csv(csv_files['providers'])
    receivers_df = pd.read_csv(csv_files['receivers'])
    food_listings_df = pd.read_csv(csv_files['food_listings'])
    claims_df = pd.read_csv(csv_files['claims'])
    
    datasets = {
        'providers': providers_df,
        'receivers': receivers_df,
        'food_listings': food_listings_df,
        'claims': claims_df
    }
    
    print("\n✅ Data loaded successfully!")
    print(f"Providers: {len(providers_df)} records")
    print(f"Receivers: {len(receivers_df)} records")
    print(f"Food Listings: {len(food_listings_df)} records")
    print(f"Claims: {len(claims_df)} records")
    
except Exception as e:
    print(f"\n❌ Error loading data: {e}")
    print("Please check:")
    print("1. File names are exactly: providers_data.csv, receivers_data.csv, food_listings_data.csv, claims_data.csv")
    print("2. Files are in the data/raw/ folder")
    print("3. Files are not corrupted")

# Cell 3: Quick Data Inspection (only if data loaded successfully)
if 'food_listings' in datasets:
    food_listings_df = datasets['food_listings']
    
    print("\n🔍 FOOD LISTINGS DATA INSPECTION:")
    print("="*50)
    print(f"Shape: {food_listings_df.shape}")
    print(f"\nColumns: {list(food_listings_df.columns)}")
    print(f"\nData types:\n{food_listings_df.dtypes}")
    print(f"\nFirst 3 rows:")
    print(food_listings_df.head(3))
    print(f"\nMissing values:\n{food_listings_df.isnull().sum()}")

# Cell 4: Data Cleaning Functions (same as before)
def clean_text_column(series):
    """Clean text columns - remove extra spaces, handle NaN"""
    return series.astype(str).str.strip().str.replace('nan', '').str.title()

def standardize_food_type(food_type):
    """Standardize food type categories"""
    if pd.isna(food_type) or str(food_type).lower() in ['nan', '', 'none']:
        return 'Vegetarian'  # Default
    
    food_type = str(food_type).strip().lower()
    
    if 'vegan' in food_type:
        return 'Vegan'
    elif any(word in food_type for word in ['non-veg', 'non vegetarian', 'meat', 'chicken', 'fish', 'mutton']):
        return 'Non-Vegetarian'
    else:
        return 'Vegetarian'

def standardize_meal_type(meal_type):
    """Standardize meal type categories"""
    if pd.isna(meal_type) or str(meal_type).lower() in ['nan', '', 'none']:
        return 'Snacks'  # Default
    
    meal_type = str(meal_type).strip().lower()
    
    if any(word in meal_type for word in ['breakfast', 'morning']):
        return 'Breakfast'
    elif any(word in meal_type for word in ['lunch', 'afternoon']):
        return 'Lunch'
    elif any(word in meal_type for word in ['dinner', 'evening']):
        return 'Dinner'
    else:
        return 'Snacks'

def fix_expiry_date(date_str):
    """Fix and standardize expiry dates"""
    if pd.isna(date_str):
        return (datetime.now() + timedelta(days=7)).strftime('%Y-%m-%d')
    
    try:
        parsed_date = pd.to_datetime(date_str)
        if parsed_date.date() < datetime.now().date():
            parsed_date = parsed_date + timedelta(days=365)
        return parsed_date.strftime('%Y-%m-%d')
    except:
        return (datetime.now() + timedelta(days=7)).strftime('%Y-%m-%d')

print("✅ Cleaning functions defined")

# Cell 5: Process Data (only if loaded successfully)
if len(datasets) == 4:  # All datasets loaded
    
    print("\n🧹 CLEANING ALL DATA")
    print("="*40)
    
    # Clean Providers
    providers_df = datasets['providers']
    providers_clean = providers_df.copy()
    providers_clean['Name'] = clean_text_column(providers_clean['Name'])
    providers_clean['Type'] = clean_text_column(providers_clean['Type'])
    providers_clean['Address'] = clean_text_column(providers_clean['Address'])
    providers_clean['City'] = clean_text_column(providers_clean['City'])
    providers_clean['Contact'] = providers_clean['Contact'].astype(str)
    providers_clean = providers_clean.drop_duplicates(subset=['Provider_ID'])
    providers_clean = providers_clean[providers_clean['Provider_ID'] > 0]
    
    providers_final = providers_clean[['Provider_ID', 'Name', 'Type', 'Address', 'City', 'Contact']].copy()
    providers_final.columns = ['provider_id', 'name', 'type', 'address', 'city', 'contact']
    print(f"✅ Providers cleaned: {len(providers_final)} records")
    
    # Clean Receivers
    receivers_df = datasets['receivers']
    receivers_clean = receivers_df.copy()
    receivers_clean['Name'] = clean_text_column(receivers_clean['Name'])
    receivers_clean['Type'] = clean_text_column(receivers_clean['Type'])
    receivers_clean['City'] = clean_text_column(receivers_clean['City'])
    receivers_clean['Contact'] = receivers_clean['Contact'].astype(str)
    receivers_clean = receivers_clean.drop_duplicates(subset=['Receiver_ID'])
    receivers_clean = receivers_clean[receivers_clean['Receiver_ID'] > 0]
    
    receivers_final = receivers_clean[['Receiver_ID', 'Name', 'Type', 'City', 'Contact']].copy()
    receivers_final.columns = ['receiver_id', 'name', 'type', 'city', 'contact']
    print(f"✅ Receivers cleaned: {len(receivers_final)} records")
    
    # Clean Food Listings
    food_listings_df = datasets['food_listings']
    food_clean = food_listings_df.copy()
    
    print(f"Original food listings: {len(food_clean)} records")
    
    # Handle missing data
    food_clean['Food_Name'] = food_clean['Food_Name'].fillna('Food Item')
    food_clean['Food_Name'] = clean_text_column(food_clean['Food_Name'])
    
    food_clean['Quantity'] = pd.to_numeric(food_clean['Quantity'], errors='coerce')
    food_clean['Quantity'] = food_clean['Quantity'].fillna(1)
    food_clean = food_clean[food_clean['Quantity'] > 0]
    
    food_clean['Expiry_Date_Fixed'] = food_clean['Expiry_Date'].apply(fix_expiry_date)
    food_clean['Food_Type_Clean'] = food_clean['Food_Type'].apply(standardize_food_type)
    food_clean['Meal_Type_Clean'] = food_clean['Meal_Type'].apply(standardize_meal_type)
    food_clean['Location'] = clean_text_column(food_clean['Location'])
    food_clean['Provider_Type'] = clean_text_column(food_clean['Provider_Type'])
    
    # Validate Provider_IDs
    valid_provider_ids = set(providers_final['provider_id'])
    food_clean['Provider_ID_Valid'] = food_clean['Provider_ID'].isin(valid_provider_ids)
    
    if food_clean['Provider_ID_Valid'].sum() < len(food_clean) * 0.5:
        valid_ids_list = list(valid_provider_ids)
        def fix_provider_id(row):
            if row['Provider_ID_Valid']:
                return row['Provider_ID']
            else:
                return np.random.choice(valid_ids_list)
        food_clean['Provider_ID_Fixed'] = food_clean.apply(fix_provider_id, axis=1)
    else:
        food_clean = food_clean[food_clean['Provider_ID_Valid']]
        food_clean['Provider_ID_Fixed'] = food_clean['Provider_ID']
    
    food_clean = food_clean.drop_duplicates(subset=['Food_ID'])
    
    food_final = pd.DataFrame({
        'food_id': food_clean['Food_ID'],
        'food_name': food_clean['Food_Name'],
        'quantity': food_clean['Quantity'].astype(int),
        'expiry_date': food_clean['Expiry_Date_Fixed'],
        'provider_id': food_clean['Provider_ID_Fixed'],
        'provider_type': food_clean['Provider_Type'],
        'location': food_clean['Location'],
        'food_type': food_clean['Food_Type_Clean'],
        'meal_type': food_clean['Meal_Type_Clean']
    })
    
    food_final = food_final.dropna(subset=['food_id', 'food_name'])
    food_final = food_final[food_final['food_id'] > 0]
    print(f"✅ Food listings cleaned: {len(food_final)} records")
    
    # Clean Claims
    claims_df = datasets['claims']
    claims_clean = claims_df.copy()
    claims_clean['Timestamp'] = pd.to_datetime(claims_clean['Timestamp'], errors='coerce')
    claims_clean = claims_clean.dropna(subset=['Timestamp'])
    
    status_mapping = {
        'pending': 'Pending',
        'completed': 'Completed', 
        'cancelled': 'Cancelled',
        'canceled': 'Cancelled'
    }
    claims_clean['Status'] = claims_clean['Status'].str.lower().map(status_mapping).fillna('Pending')
    
    valid_food_ids = set(food_final['food_id'])
    valid_receiver_ids = set(receivers_final['receiver_id'])
    
    claims_clean = claims_clean[
        claims_clean['Food_ID'].isin(valid_food_ids) & 
        claims_clean['Receiver_ID'].isin(valid_receiver_ids)
    ]
    
    claims_clean = claims_clean.drop_duplicates(subset=['Claim_ID'])
    
    claims_final = claims_clean[['Claim_ID', 'Food_ID', 'Receiver_ID', 'Status', 'Timestamp']].copy()
    claims_final.columns = ['claim_id', 'food_id', 'receiver_id', 'status', 'timestamp']
    print(f"✅ Claims cleaned: {len(claims_final)} records")

# Cell 6: Save Cleaned Data
if len(datasets) == 4:
    print("\n💾 SAVING CLEANED DATA")
    print("="*40)
    
    try:
        providers_final.to_csv(processed_data_path / 'providers_cleaned.csv', index=False)
        print(f"✅ Saved providers_cleaned.csv: {len(providers_final)} records")
        
        receivers_final.to_csv(processed_data_path / 'receivers_cleaned.csv', index=False)
        print(f"✅ Saved receivers_cleaned.csv: {len(receivers_final)} records")
        
        food_final.to_csv(processed_data_path / 'food_listings_cleaned.csv', index=False)
        print(f"✅ Saved food_listings_cleaned.csv: {len(food_final)} records")
        
        claims_final.to_csv(processed_data_path / 'claims_cleaned.csv', index=False)
        print(f"✅ Saved claims_cleaned.csv: {len(claims_final)} records")
        
        print(f"\n🎉 ALL DATA CLEANED AND SAVED!")
        print(f"📁 Location: {processed_data_path}")
        
        # Verify files
        for filename in ['providers_cleaned.csv', 'receivers_cleaned.csv', 'food_listings_cleaned.csv', 'claims_cleaned.csv']:
            filepath = processed_data_path / filename
            if filepath.exists():
                size = filepath.stat().st_size
                print(f"✅ {filename}: {size:,} bytes")
            else:
                print(f"❌ {filename}: File not created!")
        
        print(f"\n🎯 NEXT STEPS:")
        print("1. Run: python src/database/data_loader.py")
        print("2. Run: python verify_setup.py")
        print("3. All tests should now pass!")
        
    except Exception as e:
        print(f"❌ Error saving files: {e}")
        print(f"Processed data path: {processed_data_path}")
        print(f"Path exists: {processed_data_path.exists()}")

else:
    print("❌ Cannot proceed with cleaning - data loading failed!")
    print("Please fix the CSV file paths first.")

Project root: c:\Users\mjrak\OneDrive\Desktop\local_food_wastage_management
Raw data path: c:\Users\mjrak\OneDrive\Desktop\local_food_wastage_management\data\raw
Processed data path: c:\Users\mjrak\OneDrive\Desktop\local_food_wastage_management\data\processed
Raw data path exists: True
Processed data path exists: True
Files in raw data directory:
  - claims_data.csv
  - food_listings_data.csv
  - providers_data.csv
  - receivers_data.csv

📂 LOADING DATA WITH CORRECT PATHS
Checking CSV files:
✅ providers: providers_data.csv found
✅ receivers: receivers_data.csv found
✅ food_listings: food_listings_data.csv found
✅ claims: claims_data.csv found

✅ Data loaded successfully!
Providers: 1000 records
Receivers: 1000 records
Food Listings: 1000 records
Claims: 1000 records

🔍 FOOD LISTINGS DATA INSPECTION:
Shape: (1000, 9)

Columns: ['Food_ID', 'Food_Name', 'Quantity', 'Expiry_Date', 'Provider_ID', 'Provider_Type', 'Location', 'Food_Type', 'Meal_Type']

Data types:
Food_ID           int64
Foo